In [1]:
import os

In [2]:
%pwd

'd:\\iNeuron_Projects\\machine_learning_01\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\iNeuron_Projects\\machine_learning_01'

### Make data class: 
- update the entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

###  update the configuration manager in src config

In [6]:
from MachineLearning_2023.constants import *                                 # This will import all the constants from the constants.py file
from MachineLearning_2023.utils.common import read_yaml, create_directories  # This will import the read_yaml and create_directories functions from the common.py file

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        # DataIngestionConfig is being defined as a namedtuple with four fields:
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

###  update the components

In [8]:
import os
import urllib.request as request
import zipfile
from MachineLearning_2023 import logger
from MachineLearning_2023.utils.common import get_size

In [9]:
# Class for downloading and extracting a data file
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # Method to downloads a file from a URL and saves it locally.
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    # Method to extracts the downloaded zip file into a specified directory.
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


### Update the pipeline

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e

[2023-12-20 15:25:47,072: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-20 15:25:47,076: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-20 15:25:47,079: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-20 15:25:47,083: INFO: common: created directory at: artifacts]
[2023-12-20 15:25:47,085: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-20 15:25:50,590: INFO: 3920769660: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f9ebb3f117bc8c3e2e56a198425ff3233303a3d0d1025b1ddf31ec815a161874"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 487C:2C88:103366:1354D2:6582D3DD
Accept-Ranges: bytes
Date: Wed, 20 D